In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [10]:
jumlah_index = 50
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [11]:
def scrape_links(page_number,keywords):
    url = f"https://www.cnbcindonesia.com/search?query={keywords}&p={page_number}&kanal=&tipe=&date="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 12 links from page 11
Scraped 12 links from page 22
Scraped 12 links from page 49
Scraped 12 links from page 5
Scraped 12 links from page 14
Scraped 12 links from page 1
Scraped 12 links from page 2
Scraped 12 links from page 15
Scraped 12 links from page 29
Scraped 12 links from page 12
Scraped 12 links from page 6
Scraped 12 links from page 7
Scraped 12 links from page 30
Scraped 12 links from page 3
Scraped 12 links from page 13
Scraped 12 links from page 20
Scraped 12 links from page 24
Scraped 12 links from page 50
Scraped 12 links from page 31
Scraped 12 links from page 36
Scraped 12 links from page 32
Scraped 12 links from page 33
Scraped 12 links from page 10
Scraped 12 links from page 35
Scraped 12 links from page 18
Scraped 12 links from page 16
Scraped 12 links from page 34
Scraped 12 links from page 37
Scraped 12 links from page 40
Scraped 12 links from page 38
Scraped 12 links from page 27
Scraped 12 links from page 48
Scraped 12 links from page 28
Scraped 12 links

In [13]:
results = []
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[1].strip()
                author_text = author_text.split(',')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail_text"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"
            
            nama_berita_match = re.search(r'https://(.*?)/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"

            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 600


,title,keywords,author,category,date,content,nama_berita,link
0,"Anies-Cak Imin Menang, Sektor Saham Ini Perlu ...",anies,rev,Berita Research,09 September 2023 15:15,"Jakarta, CNBC Indonesia - Berbagai kebijakan y...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/research/2023090...
1,"Sosok Kuat Cawapres Ganjar, PPP Sebut Satu Nama",anies,Eqqi Syahputra,Berita,09 September 2023 16:15,"Jakarta, CNBC Indonesia - Teka-teki peta perpo...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230909131...
2,Anies Buka Suara Soal Duet dengan Ganjar di Pi...,anies,Wiji Nur Hayat,Berita,25 August 2023 06:15,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230824205...
3,"Jadi Cawapres Terkuat Prabowo, Harta Gibran Ca...",anies,Arrijal Rachman,Berita Market,09 September 2023 06:40,"Jakarta, CNBC Indonesia - Menjelang batas wakt...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/market/202309090...
4,"Duet Anies-Ganjar Makin Heboh, Tinggal Tunggu ...",anies,Wiji Nur Hayat,Berita,24 August 2023 06:15,"Jakarta, CNBC Indonesia - Tiba-tiba muncul wac...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230823224...
5,Terkuak! Ternyata Ini Sosok Cawapres Anies,anies,Wiji Nur Hayat,Berita,04 July 2023 06:20,"Jakarta, CNBC Indonesia - Sosok calon wakil pr...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230703185...
6,Prabowo & Ganjar Siap-siap! Efek Ini Buat Anie...,anies,Arrijal Rachman,Berita,09 September 2023 08:30,"Jakarta, CNBC Indonesia - Calon presiden Anies...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230909074...
7,"Jangan Kaget, Ini Survei Terbaru Ganjar Vs Ani...",anies,Tim Redaksi,Berita,23 August 2023 14:37,"Jakarta, CNBC Indonesia - Saiful Mujani Resear...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230823143...
8,"Jokowi Termuda, Rata-Rata Umur Capres RI Masuk...",anies,Tasya Natalia,Berita Research,24 August 2023 05:40,"Jakarta, CNBC Indonesia - Persoalan batasan um...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/research/2023082...
9,"Cak Imin Diperiksa KPK, Mahfud MD: Saya Juga P...",anies,Rosseno Aji,Berita,09 September 2023 15:30,"Jakarta, CNBC Indonesia - Menteri Koordinator ...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230909131...


In [16]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 564


,title,keywords,author,category,date,content,nama_berita,link
0,"Anies-Cak Imin Menang, Sektor Saham Ini Perlu ...",anies,rev,Berita Research,09 September 2023 15:15,"Jakarta, CNBC Indonesia - Berbagai kebijakan y...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/research/2023090...
1,"Sosok Kuat Cawapres Ganjar, PPP Sebut Satu Nama",anies,Eqqi Syahputra,Berita,09 September 2023 16:15,"Jakarta, CNBC Indonesia - Teka-teki peta perpo...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230909131...
2,Anies Buka Suara Soal Duet dengan Ganjar di Pi...,anies,Wiji Nur Hayat,Berita,25 August 2023 06:15,"Jakarta, CNBC Indonesia - Bakal Calon Presiden...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230824205...
3,"Jadi Cawapres Terkuat Prabowo, Harta Gibran Ca...",anies,Arrijal Rachman,Berita Market,09 September 2023 06:40,"Jakarta, CNBC Indonesia - Menjelang batas wakt...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/market/202309090...
4,"Duet Anies-Ganjar Makin Heboh, Tinggal Tunggu ...",anies,Wiji Nur Hayat,Berita,24 August 2023 06:15,"Jakarta, CNBC Indonesia - Tiba-tiba muncul wac...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230823224...
5,Terkuak! Ternyata Ini Sosok Cawapres Anies,anies,Wiji Nur Hayat,Berita,04 July 2023 06:20,"Jakarta, CNBC Indonesia - Sosok calon wakil pr...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230703185...
6,Prabowo & Ganjar Siap-siap! Efek Ini Buat Anie...,anies,Arrijal Rachman,Berita,09 September 2023 08:30,"Jakarta, CNBC Indonesia - Calon presiden Anies...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230909074...
7,"Jangan Kaget, Ini Survei Terbaru Ganjar Vs Ani...",anies,Tim Redaksi,Berita,23 August 2023 14:37,"Jakarta, CNBC Indonesia - Saiful Mujani Resear...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230823143...
8,"Jokowi Termuda, Rata-Rata Umur Capres RI Masuk...",anies,Tasya Natalia,Berita Research,24 August 2023 05:40,"Jakarta, CNBC Indonesia - Persoalan batasan um...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/research/2023082...
10,Anies Bakal Salip Prabowo dan Ganjar? Efek Ini...,anies,Wiji Nur Hayat,Berita,08 September 2023 12:41,"Jakarta, CNBC Indonesia - Para kandidat calon ...",www.cnbcindonesia.com,https://www.cnbcindonesia.com/news/20230908111...


Save File

In [17]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/cnbc_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')